## Data Preprocessing

This dataset is trying to predict whether or not a customer stays with a bank over time.

In [16]:
# Load in the data
import os
import pandas as pd

DATASET_PATH = "datasets/"
DATASET_NAME = "Churn_Modelling.csv"
DATASET_URL = DATASET_PATH + DATASET_NAME

def fetch_data(dataset_url=DATASET_URL, dataset_path=DATASET_PATH):
    if not os.path.isdir(dataset_path):
        os.makedirs(dataset_path)

dataset = fetch_data()

def load_data(dataset_path=DATASET_PATH, dataset_name=DATASET_NAME):
    csv_path = os.path.join(dataset_path, dataset_name)
    return pd.read_csv(csv_path)

dataset = load_data()

In [17]:
# Create training and test sets
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [18]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#Encode Countries
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

#Encode Gender
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:] # Avoid dummy variable trap

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [20]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Build ANN
This model will use keras to build the neural network

In [21]:
import keras
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()

# Add input layer and first hidden layer - output_dim set as 11 (ind var) + 1 (dep var) = 12 / 2 = 6
classifier.add(Dense(units = 6, kernel_initializer='uniform', activation='relu', input_dim=11))

# Add second layer (not necessary for this dataset but useful for template)
classifier.add(Dense(units = 6, kernel_initializer='uniform', activation='relu'))

# Add output layer, for an output with multiple outputs change the output_dim and the activation to softmax
classifier.add(Dense(units = 1, kernel_initializer='uniform', activation='sigmoid'))

classifier.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 1s 178us/step - loss: 0.4851 - acc: 0.7962
Epoch 2/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.4184 - acc: 0.8180
Epoch 3/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.4056 - acc: 0.8290
Epoch 4/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.3943 - acc: 0.8290
Epoch 5/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.3873 - acc: 0.8300
Epoch 6/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.3810 - acc: 0.8315
Epoch 7/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.3765 - acc: 0.8372
Epoch 8/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.3723 - acc: 0.8470
Epoch 9/100
8000/8000 [==============================] - 1s 142us/step - loss: 0.3687 - acc: 0.8471
Epoch 10/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.3664 - acc: 0.8475

8000/8000 [==============================] - 1s 143us/step - loss: 0.3375 - acc: 0.8604
Epoch 83/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.3376 - acc: 0.8627
Epoch 84/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.3375 - acc: 0.8626
Epoch 85/100
8000/8000 [==============================] - 1s 142us/step - loss: 0.3378 - acc: 0.8615
Epoch 86/100
8000/8000 [==============================] - 1s 142us/step - loss: 0.3366 - acc: 0.8609
Epoch 87/100
8000/8000 [==============================] - 1s 142us/step - loss: 0.3370 - acc: 0.8622
Epoch 88/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.3366 - acc: 0.8612
Epoch 89/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.3378 - acc: 0.8614
Epoch 90/100
8000/8000 [==============================] - 1s 142us/step - loss: 0.3386 - acc: 0.8605
Epoch 91/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.3367 - acc: 0.8622
Epo

In [25]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [31]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

cf_mat = confusion_matrix(y_test, y_pred)
tp = cf_mat[0,0]
tn = cf_mat[1,1]
fp = cf_mat[1,0]
fn = cf_mat[0,1]
total = tp+tn+fp+fn

#I'm cheating b/c test set is 100 data pts.
print("Accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Precision: %.2f" % precision_score(y_test, y_pred))
print('Recall: %.2f' % recall_score(y_test, y_pred))
print('F1 Score: %.2f' % f1_score(y_test, y_pred))
print("Percentage True Positive: %.2f" % ((tp/total)*100))
print("Percentage True Negative: %.2f" % ((tn/total)*100))
confusion_matrix(y_test, y_pred)

Accuracy: 0.86
Precision: 0.73
Recall: 0.51
F1 Score: 0.60
Percentage True Positive: 75.90
Percentage True Negative: 10.35


array([[1518,   77],
       [ 198,  207]])